In [ ]:
# %load_ext lab_black

In [ ]:
# Set for local or colab

import os
from os.path import join
import shutil
import sys

ASK_TO_DELETE_LOG_FOLDERS = True


def check_create_folder(path: str, ask_to_rm_if_exists=ASK_TO_DELETE_LOG_FOLDERS):
    if os.path.exists(path):
        if ask_to_rm_if_exists:
            response = input(
                f"<{path}>: Already exists.\n\nWrite 'del' if you wish to delete other wise press any key"
            )
            if response.lower() == "del":
                print(f"Deleting: {path}")
                shutil.rmtree(path)

                os.makedirs(path)
    else:
        os.makedirs(path)


# Check if running in colab
IN_COLAB = "google.colab" in sys.modules

# Project defaults
if IN_COLAB:
    print("ENVIRONMENT: Colab")

    # Mount drive
    from google.colab import drive

    drive.mount("/content/drive")

    # Set the project directory
    PROJECT_FOLDER = "/content/drive/MyDrive/w266/w266-project-carlos"

    # Install dependencies
    !pip install -q transformers datasets pytorch-lightning wandb
else:
    print("ENVIRONMENT: Local")
    # Set the project directory
    PROJECT_FOLDER = "/user/w266/w266-project-carlos"

os.chdir(PROJECT_FOLDER)

# FOLDERS
DATASET_FOLDER = join(PROJECT_FOLDER, "dataset")
CHECKPOINT_FOLDER = join(PROJECT_FOLDER, "checkpoints")
MODEL_FOLDER = join(PROJECT_FOLDER, "saved_models")
LOGGER_FOLDER = join(PROJECT_FOLDER, "logger")

check_create_folder(CHECKPOINT_FOLDER)
check_create_folder(MODEL_FOLDER)
check_create_folder(LOGGER_FOLDER)

print(f"Working directory is: {os.getcwd()}")

In [ ]:
import numpy as np
import pandas as pd
import copy
from datasets import Dataset
from transformers import RobertaTokenizer
import torch
from torch.utils.data import DataLoader
from transformers import (
    T5ForConditionalGeneration,
    AdamW,
    get_linear_schedule_with_warmup,
)
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks import (
    EarlyStopping,
    LearningRateMonitor,
    TQDMProgressBar,
    Callback,
)

import math

np.random.seed(123)

### Load `csv` data as a single `dataframe`

In [ ]:
def combine_csv_files(csv_paths: list[str], shuffle=False):
    """
    Combine csv data into a single dataframe and checks for duplicate records.

    """
    for i, path in enumerate(csv_paths):
        df = pd.read_csv(path)
        columns = df.columns

        print(f"Number of records in {path}: {df.shape[0]}")

        if i == 0:
            df_full = df
            columns_base = columns
        else:
            if not np.array_equal(columns, columns_base):
                raise (Exception("Columns do not match"))

            total_records = df_full.shape[0] + df.shape[0]

            df_full = (
                pd.concat([df_full, df]).drop_duplicates(columns).reset_index(drop=True)
            )

            records_dropped = total_records - df_full.shape[0]

            print(f"-> Merged!!, {records_dropped} duplicates were found and dropped")

        print("")

    if shuffle:
        shuffled_indices = np.random.permutation(np.arange(df_full.shape[0]))
        df_full = df_full.iloc[shuffled_indices, :].reset_index(drop=True)

    print(f"A total of {df_full.shape[0]} recrods were loaded")
    return df_full


df_dataset = combine_csv_files(
    [
        join(DATASET_FOLDER, "train.csv"),
        join(DATASET_FOLDER, "dev.csv"),
        join(DATASET_FOLDER, "test.csv"),
    ],
    shuffle=False,
)

df_dataset.head()

In [ ]:
print(f"Total records {df_dataset.shape[0]}")
print("Records with empyt rows:")
display(df_dataset[df_dataset.isna().values])


print("Dropping NaN...")
df_dataset.dropna(inplace=True)
print(f"New total records {df_dataset.shape[0]}")

### Create the datasets

In [ ]:
# TOTAL_DATASET_SIZE = df_dataset.shape[0]
TOTAL_DATASET_SIZE = 10


def get_indexes(total_indices, splits=[0.70, 0.10, 0.20]):
    start_index = 0
    indices = []
    groups = len(splits)

    for i, split in enumerate(splits):
        count = int(total_indices * split)
        end_index = start_index + count

        print(f"Group {i} > {start_index}:{end_index-1}, {count}")

        if i + 1 < groups:
            indices.append(np.arange(start_index, end_index))
        else:
            indices.append(np.arange(start_index, total_indices))

        start_index = end_index

    return indices


indces = get_indexes(TOTAL_DATASET_SIZE, splits=[0.60, 0.20, 0.20])

train_dataset = Dataset.from_pandas(df_dataset.iloc[indces[0]], split="train")
val_dataset = Dataset.from_pandas(df_dataset.iloc[indces[1]], split="validation")
test_dataset = Dataset.from_pandas(df_dataset.iloc[indces[2]], split="train")

display(train_dataset)
display(val_dataset)
display(test_dataset)

display(train_dataset[0])

### Pre-process and tokenize the data

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained("Salesforce/codet5-small")

prefix = "Generate vega_zero code: "
max_input_length = 100
max_target_length = 100

In [ ]:
def preprocess_examples(examples, add_db_info=False):
    """
    This function process the input and targets (labels)

    Inputs:
    - Adds a prefix question (for t5)
    - Tokenizes the input

    Targets (labels):
    - Tokenizes
    - Replaces the padding token index from 0 to -100
    """
    questions = examples["question"]  # inputs
    queries = examples["query"]  # targets

    inputs = [prefix + question for question in questions]

    if add_db_info:
        pass

    # Tokenize the inputs
    model_inputs = tokenizer(
        inputs, max_length=max_input_length, padding="max_length", truncation=True
    )

    # Tokenize the targets
    labels = tokenizer(
        queries, max_length=max_target_length, padding="max_length", truncation=True
    ).input_ids

    # important: we need to replace the index of the padding tokens by -100
    # such that they are not taken into account by the CrossEntropyLoss
    labels_with_ignore_index = []
    for labels_example in labels:
        labels_example = [label if label != 0 else -100 for label in labels_example]
        labels_with_ignore_index.append(labels_example)

    model_inputs["labels"] = labels_with_ignore_index

    return model_inputs


# Map the function to each dataset
train_dataset = train_dataset.map(preprocess_examples, batched=True)
val_dataset = val_dataset.map(preprocess_examples, batched=True)
test_dataset = test_dataset.map(preprocess_examples, batched=True)

columns = ["input_ids", "attention_mask", "labels"]

# This sets `__getitem__` return format (type and columns). The data formatting is applied on-the-fly.
# `__getitem__` is what pulls the batches during training
train_dataset.set_format(type="torch", columns=columns)
val_dataset.set_format(type="torch", columns=columns)
test_dataset.set_format(type="torch", columns=columns)

print("Training")
print(train_dataset)
print("*" * 100)

print("Validation")
print(val_dataset)
print("*" * 100)

print("Test")
print(test_dataset)

# Without the `.set_format`, this would get you all the columns
print(train_dataset[0].keys())

### Check that the previous is working as intended

In [ ]:
sample_dataloader = DataLoader(val_dataset, batch_size=4)

batch = next(iter(sample_dataloader))

print(f"The keys for each batch are:")
print(batch.keys())
print("*" * 100)

print("Input token ids:")
print(batch["input_ids"][0])
print("*" * 100)

print("Decoded input tokens:")
print(tokenizer.decode(batch["input_ids"][0]))
print("*" * 100)

print("Label token ids:")
labels = batch["labels"][0]
print(labels)
print("*" * 100)

print("Decoded label tokens:")
print(tokenizer.decode([label for label in labels if label != -100]))

### Declare and tune model

In [ ]:
# hyper parameters
num_epochs = 5
batch_size = 3
test_batch_size = 2
learning_rate = 5e-5
# warmup_steps = 1000

# Calculated values
training_set_len = len(train_dataset)
batches_per_epoch = math.ceil(training_set_len / batch_size)
total_training_steps = num_epochs * batches_per_epoch
warmup_steps = (
    int(np.round(num_epochs / 3)) * batches_per_epoch
)  # after a third of the epocs

# Extra note-only parameters
val_set_len = len(val_dataset)
test_set_len = len(test_dataset)

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, batch_size=test_batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=test_batch_size)

assert len(train_dataloader) == batches_per_epoch

print("Training stats:")
print(f"learning_rate: {learning_rate}")
pd.DataFrame(
    index=[
        "TOTAL Samples",
        "training_set_len",
        "val_set_len",
        "test_set_len",
        "num_epochs",
        "batch_size",
        "batches_per_epoch",
        "total_training_steps",
        "warmup_steps",
    ],
    columns=["training_value"],
    data=[
        training_set_len + val_set_len + test_set_len,
        training_set_len,
        val_set_len,
        test_set_len,
        num_epochs,
        batch_size,
        batches_per_epoch,
        total_training_steps,
        warmup_steps,
    ],
)

In [ ]:
class EpochProgressBar(TQDMProgressBar):
    """
    This extends the base progress bar to not overwrite the progress bar
    and show its history.
    """

    def on_train_epoch_end(self, trainer="pl.Trainer", pl_module="pl.LightningModule"):
        super().on_train_epoch_end(trainer=trainer, pl_module=pl_module)
        print("\n")


class HistoryCallback(Callback):
    """PyTorch Lightning metric callback."""

    def __init__(self):
        super().__init__()
        self.history = []

    def on_train_epoch_end(self, trainer, pl_module):
        each_me = copy.deepcopy(trainer.callback_metrics)
        self.history.append(each_me)

    def history_dataframe(self):
        return pd.DataFrame(self.history).astype(np.float32)


class CodeT5(pl.LightningModule):
    def __init__(
        self,
        num_epochs,
        batch_size,
        lr,
        training_set_len,
        total_training_steps,
        warmup_steps,
    ):
        super().__init__()

        # Model or layers
        self.model = T5ForConditionalGeneration.from_pretrained(
            "Salesforce/codet5-small"
        )

        self.save_hyperparameters()

        # Debuggin vars(not needed for training)
        self.training_steps_completed = 0.0

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.model(
            input_ids=input_ids, attention_mask=attention_mask, labels=labels
        )
        return outputs

    def common_step(self, batch, batch_idx):
        # `batch` is a dictionary, the '**' before batch
        # allows the 'forward step' to directly unpack the dictionary
        outputs = self(**batch)

        # The pretrained model aut calcs the loss
        loss = outputs.loss

        return loss

    def training_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)
        self.log(
            "training_loss",
            loss,
            on_step=False,
            on_epoch=True,
            prog_bar=True,
            logger=True,
        )

        last_lr = self.lr_schedulers().get_last_lr()[0]
        self.log(
            "learning_rate",
            last_lr,
            on_step=False,
            on_epoch=True,
            prog_bar=True,
            logger=True,
        )

        self.training_steps_completed += 1.0  # logger don't like ints
        self.log(
            "training_steps_completed",
            self.training_steps_completed,
            on_step=False,
            on_epoch=True,
            prog_bar=True,
            logger=True,
        )

        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)
        self.log(
            "validation_loss",
            loss,
            on_step=False,
            on_epoch=True,
            prog_bar=True,
            logger=True,
        )

        return loss

    def test_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)
        return loss

    def configure_optimizers(self):
        # create optimizer
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.hparams.lr)

        # create learning rate scheduler
        lr_scheduler = {
            "scheduler": get_linear_schedule_with_warmup(
                optimizer,
                num_warmup_steps=self.hparams.warmup_steps,
                num_training_steps=self.hparams.total_training_steps,
            ),
            "name": "learning_rate",
            "interval": "step",
            "frequency": 1,
        }

        return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}

    def train_dataloader(self):
        return train_dataloader

    def val_dataloader(self):
        return val_dataloader

    def test_dataloader(self):
        return test_dataloader


model = CodeT5(
    num_epochs=num_epochs,
    batch_size=batch_size,
    lr=learning_rate,
    training_set_len=training_set_len,
    total_training_steps=total_training_steps,
    warmup_steps=warmup_steps,
)

# -----------------------------------------------------------------------
# Trainer section
# -----------------------------------------------------------------------

# Logger
logger = CSVLogger(save_dir=LOGGER_FOLDER, name="My_Logger")

# Callbacks
progress_bar = EpochProgressBar()
history = HistoryCallback()

# Trainer
trainer = Trainer(
    default_root_dir=CHECKPOINT_FOLDER,
    callbacks=[progress_bar, history],
    logger=logger,
    max_epochs=num_epochs,
)

trainer.fit(model)

history.history_dataframe()

In [ ]:
model.model.save_pretrained(MODEL_FOLDER)